1) gangnamOutput-modify.csv에서 지역만 얻어내기

In [2]:
import pandas as pd
data = pd.read_csv('/Users/junho/Hadoop-Classification/data/crawling/gangnamOutput-modify.csv')
data.describe()
unique_regions = data['region'].unique().tolist()
restaurant_list = data['name'].tolist()
unique_regions = data['region'].str.replace('서울 ', '', regex=False).unique().tolist()
print(unique_regions)
print(restaurant_list)

['마포', '영등포', '성동', '강북', '관악', '송파', '강서', '용산', '강남', '광진', '서초', '서대문', '동작', '양천', '금천', '은평', '중구', '성북', '종로', '동대문', '중랑', '강동', '노원', '도봉']
['살쾡이', '모에루 문래본점', 'HDD피자', '와와 쪽갈비 수유점', '소나무뜰끌림부추삼겹살', '백정집', '이자카야 토라 수유점', '마미떡 송파점', '제이클레프', '장효서울', '중화식', '수다 청담점', '153 구포국수 군자점', '하우마라탕', '우리돼지 솥뚜껑 삼겹살', '등촌샤브칼국수 사당역점', '영동소곱창', '루프탑미드', '하남돼지집 서울대입구역점', '신용산진배기고기집', '트리플에이커피 봉천e편한세상점', 'HDD판다', '쿠라우니', '청담오후', '포도젝트', '시온텐동', '수다 서래점', '신촌양꼬치', '막홀릭하우스', '임실순대국', '우뚝1984', '카페아뜨레', '더 가즈믹 테이블', '카페 더하리', '하마루', '주술소', '꽃보다크레페', '오목교곱창 목동본점', '라모따', '팔러 서울', '이자카야 명호', '제일곱창 연신내직영점', '트리블리 와인바', '유타로', '이천면옥함흥냉면', '갈매기의꿈', '하라고지페', '스시신조', '매료쪽갈비찜 송파본점', '곰팔이생고기,돼지갈비', '온량(장위동)', '호랭이술상', '김봉남포장마차 가로수길근처점', '미스터쭈꾸미', '빅파이브 종로점', '이태원교집합', '대물장어', '사조참치 장한평점', '향만루', '더양평해장국 성수역점', '조옥당 약수본점', '오목교별장', '희객 2호점', '켄홈 방이점', 'After Work33', '연남토마 성수점', '주한상', '우미마토', '달길포차', '뜸들이다 시흥점', '스트라이프', '골짝마을 닭한마리', '오늘튀김', '이여곰탕 마포점', '언필드', '수다 대치점', '익선취향', '이수가 원조', '갈치호', '광어연어', '일번지

2. 라이브러리

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv

In [13]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# 스크롤 함수 정의
def scroll_down(driver, n):
    for _ in range(n):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

# 크롤링 시작
for region in unique_regions:
    region_name = f"서울_{region}_식당.csv"
    folder_path = f"/Users/junho/Hadoop-Classification/data/crawling"
    file_path = os.path.join(folder_path, region_name)
    
    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 이미 존재하면 스킵
    if os.path.exists(file_path):
        print(f"File {file_path} already exists. Skipping...\n")
        continue
    
    region_restaurants = [restaurant for restaurant, reg in zip(restaurant_list, data['region']) if reg == f"서울 {region}"]
    
    url_list = []
    title_list = []
    
    for restaurant in region_restaurants:
        # 크롬 드라이버 설정 및 실행
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('/Users/junho/bdp/chromedriver-mac-arm64/chromedriver')
        chrome_options.add_argument("headless")
        driver = webdriver.Chrome(options=chrome_options)
        
        try:
            # 네이버 검색 페이지 접속
            driver.get("https://search.naver.com")
            time.sleep(1)
            
            # 검색창에 지역 + 식당 이름 입력 및 검색
            search_box = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            search_box.send_keys(f"{region} {restaurant}")
            search_box.submit()
            time.sleep(1)
            
            driver.find_element(By.LINK_TEXT, "블로그").click()
            time.sleep(1)
            
            # 검색 결과 크롤링
            scroll_down(driver, n=1)
            titles = driver.find_elements(By.CSS_SELECTOR, '#SE-0a7e6dac-92b8-11ef-8990-ff1d0d22bde5 > div > div > div.pcol1 > div')
            articles = driver.find_elements(By.CSS_SELECTOR, "a.title_link")
            
            for article in articles:
                title = article.text
                url = article.get_attribute("href")
                
                # URL과 제목 저장
                url_list.append(url)
                title_list.append(title)
        
        except Exception as e:
            print(f"Error while processing {restaurant}: {e}")
        
        finally:
            driver.quit()
    
    # 크롤링 데이터 저장
    df = pd.DataFrame({'url': url_list, 'title': title_list})
    df.to_csv(file_path, index=False)
    print(f"Saved {file_path}")

print("Crawling completed.")


File /Users/junho/Hadoop-Classification/data/crawling/서울_마포_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_영등포_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_성동_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_강북_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_관악_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_송파_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_강서_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_용산_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_강남_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification/data/crawling/서울_광진_식당.csv already exists. Skipping...

File /Users/junho/Hadoop-Classification

In [5]:
import os
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By

# Chrome WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('/Users/junho/bdp/chromedriver-mac-arm64/chromedriver')
chrome_options.add_argument('headless')  # Headless 모드 설정

# 저장 경로 설정
result_folder_path = "/Users/junho/Hadoop-Classification/data/crawling/blog-review"

# 결과 폴더 생성
os.makedirs(result_folder_path, exist_ok=True)

# 크롤링 시작
for region in unique_regions:
    # URL CSV 파일 읽기
    url_csv_path = os.path.join(f"/Users/junho/Hadoop-Classification/data/crawling", f"서울_{region}_식당.csv")
    if not os.path.exists(url_csv_path):
        print(f"File {url_csv_path} not found. Skipping...\n")
        continue
    
    url_df = pd.read_csv(url_csv_path)
    url_list = url_df['url'].tolist()
    
    # 지역별 데이터 저장 리스트
    region_data = []

    for url in url_list:
        m_url = "https://m." + url.replace("https://", "")
        # Selenium WebDriver 실행
        driver = webdriver.Chrome(options=chrome_options)
        try:
            # URL 접속
            driver.get(m_url)
            time.sleep(2)
            
            # 제목 및 본문 크롤링
            title = ""
            content = ""
            try:
                # Title 크롤링
                title_element = driver.find_element(By.CSS_SELECTOR, ".se-title-text")
                title = title_element.text.strip()
            except Exception as e:
                print(f"Error fetching title for {m_url}: {e}")
            
            try:
                # Content 크롤링
                content_element = driver.find_element(By.CSS_SELECTOR, ".se-main-container")
                content = content_element.text.strip().replace("\n", " ")  # 줄바꿈 제거
            except Exception as e:
                print(f"Error fetching content for {m_url}: {e}")
            
            # 데이터 추가
            region_data.append({"Title": title, "Content": content})
            print(f"Added data for {m_url}")

        except Exception as e:
            print(f"Error while processing {m_url}: {e}")
        
        finally:
            driver.quit()
    
    # 지역별 데이터 저장
    if region_data:
        region_file_path = os.path.join(result_folder_path, f"{region}_blog_review.csv")
        region_df = pd.DataFrame(region_data)
        region_df.to_csv(
            region_file_path,
            index=False,
            encoding="utf-8",
            quotechar='"',  # 인용부호 사용
            quoting=csv.QUOTE_ALL  # 모든 필드를 인용부호로 감쌈
        )
        print(f"Saved data for region '{region}' to {region_file_path}")
    else:
        print(f"No data collected for region '{region}'.")

print("Crawling completed.")


Added data for https://m.blog.naver.com/jaegizang/222961713235
Added data for https://m.blog.naver.com/dev-dev/223596636795
Added data for https://m.blog.naver.com/jwda1004/223303361788
Added data for https://m.blog.naver.com/tina1115/223170406217
Added data for https://m.blog.naver.com/moment1819/223202688980
Added data for https://m.blog.naver.com/dbfla010529/223120698259
Added data for https://m.blog.naver.com/hanseul1919/223161706783
Added data for https://m.blog.naver.com/mkj5192/223404827999
Added data for https://m.blog.naver.com/lynaseveryday/223303650632
Added data for https://m.blog.naver.com/ddinga_log/223219607697
Added data for https://m.blog.naver.com/gulungi/223503775996
Added data for https://m.blog.naver.com/kyt4038/223045237542
Added data for https://m.blog.naver.com/sdk8121/223388611723
Added data for https://m.blog.naver.com/mbcswallow/222413953194
Added data for https://m.blog.naver.com/agdal2/223532117713
Added data for https://m.blog.naver.com/y00storie/223629500